# Test of module myutil.cache
Documentation: [here](../doc/_build/html/cache.html)

In [1]:
%autoreload 2
#!rm -f cache.dir/*

## Base caching

In [2]:
from myutil import loggingBasicConfig
loggingBasicConfig(level='INFO')
from myutil.demo.cache import simplefunc as f
f.cache.clear(); f.cache.resize(10)
f.cache.info()

CellInfo(signature='myutil.demo.cache.simplefunc', hits=0, misses=0, maxsize=10, currsize=0)

All cache lookups miss (if cache was cleared):

In [3]:
for i in range(15): f(x=1,y=i)

INFO:myutil.cache:Cache</home/andreoli/src/PYTOOLS/demo/cache.dir:myutil.demo.cache.simplefunc(x,y,-z)> MISS(1)
INFO:myutil.cache:Cache</home/andreoli/src/PYTOOLS/demo/cache.dir:myutil.demo.cache.simplefunc(x,y,-z)> MISS(2)
INFO:myutil.cache:Cache</home/andreoli/src/PYTOOLS/demo/cache.dir:myutil.demo.cache.simplefunc(x,y,-z)> MISS(3)
INFO:myutil.cache:Cache</home/andreoli/src/PYTOOLS/demo/cache.dir:myutil.demo.cache.simplefunc(x,y,-z)> MISS(4)
INFO:myutil.cache:Cache</home/andreoli/src/PYTOOLS/demo/cache.dir:myutil.demo.cache.simplefunc(x,y,-z)> MISS(5)
INFO:myutil.cache:Cache</home/andreoli/src/PYTOOLS/demo/cache.dir:myutil.demo.cache.simplefunc(x,y,-z)> MISS(6)
INFO:myutil.cache:Cache</home/andreoli/src/PYTOOLS/demo/cache.dir:myutil.demo.cache.simplefunc(x,y,-z)> MISS(7)
INFO:myutil.cache:Cache</home/andreoli/src/PYTOOLS/demo/cache.dir:myutil.demo.cache.simplefunc(x,y,-z)> MISS(8)
INFO:myutil.cache:Cache</home/andreoli/src/PYTOOLS/demo/cache.dir:myutil.demo.cache.simplefunc(x,y,-z)> 

In [4]:
f.cache.info()

CellInfo(signature='myutil.demo.cache.simplefunc', hits=0, misses=15, maxsize=10, currsize=10)

In [5]:
f.cache

All cache lookups hit, except last one (for i=15):

In [6]:
for i in range(5,10): f(y=i,x=1)
for i in range(12,16): f(x=1,y=i)

INFO:myutil.cache:Cache</home/andreoli/src/PYTOOLS/demo/cache.dir:myutil.demo.cache.simplefunc(x,y,-z)> HIT(6)
INFO:myutil.cache:Cache</home/andreoli/src/PYTOOLS/demo/cache.dir:myutil.demo.cache.simplefunc(x,y,-z)> HIT(7)
INFO:myutil.cache:Cache</home/andreoli/src/PYTOOLS/demo/cache.dir:myutil.demo.cache.simplefunc(x,y,-z)> HIT(8)
INFO:myutil.cache:Cache</home/andreoli/src/PYTOOLS/demo/cache.dir:myutil.demo.cache.simplefunc(x,y,-z)> HIT(9)
INFO:myutil.cache:Cache</home/andreoli/src/PYTOOLS/demo/cache.dir:myutil.demo.cache.simplefunc(x,y,-z)> HIT(10)
INFO:myutil.cache:Cache</home/andreoli/src/PYTOOLS/demo/cache.dir:myutil.demo.cache.simplefunc(x,y,-z)> HIT(13)
INFO:myutil.cache:Cache</home/andreoli/src/PYTOOLS/demo/cache.dir:myutil.demo.cache.simplefunc(x,y,-z)> HIT(14)
INFO:myutil.cache:Cache</home/andreoli/src/PYTOOLS/demo/cache.dir:myutil.demo.cache.simplefunc(x,y,-z)> HIT(15)
INFO:myutil.cache:Cache</home/andreoli/src/PYTOOLS/demo/cache.dir:myutil.demo.cache.simplefunc(x,y,-z)> MISS

In [7]:
f.cache

In [8]:
f.cache.resize(4)
f.cache

INFO:myutil.cache:Cache</home/andreoli/src/PYTOOLS/demo/cache.dir:myutil.demo.cache.simplefunc(x,y,-z)> OVERFLOW(6)


## Error management

In [9]:
from myutil.demo.cache import longfunc
longfunc.cache.clear()
longfunc.cache.info()
class timing:
    def __enter__(self): from time import perf_counter; self.time = perf_counter(); return self
    def __exit__(self,*a): from time import perf_counter; self.time = perf_counter()-self.time

Cache lookup misses (if cache was cleared).
Execution lasts 3 seconds and raises exception, which is cached.
On second iteration, cache hits and exception is directly raised.
Note that cache values which are exceptions are marked with a star in the size column.

In [10]:
for i in range(1,3):
    print('Iteration',i,end='...',flush=True)
    with timing() as T:
        try: v = longfunc(None,3)
        except Exception as e: v = 'Exception[{}]'.format(e)
    print('result:',v,'({:.01f}sec)'.format(T.time),flush=True)

Iteration 1...

INFO:myutil.cache:Cache</home/andreoli/src/PYTOOLS/demo/cache.dir:myutil.demo.cache.longfunc(x,-delay)> MISS(17)


result: Exception[longfunc error] (3.1sec)
Iteration 2...

INFO:myutil.cache:Cache</home/andreoli/src/PYTOOLS/demo/cache.dir:myutil.demo.cache.longfunc(x,-delay)> HIT(17)


result: Exception[longfunc error] (0.0sec)


In [11]:
longfunc.cache

Now, interrupt execution for 42,5 in iteration 1. The ``KeyboardInterrupt`` exception is cached. The ``clear_error`` call will clear that cached value so iteration 2 will complete successufully.

In [12]:
for i in range(1,3):
    for x,t in (36,2),(42,5):
        print('Iteration',i,(x,t),end='...',flush=True)
        with timing() as T:
            try: v = longfunc(x,t)
            except BaseException as e: v = 'Exception[{}]'.format(e)
        print('result:',v,'({:.01f}sec)'.format(T.time),flush=True)
    longfunc.cache.clear_error()

Iteration 1 (36, 2)...

INFO:myutil.cache:Cache</home/andreoli/src/PYTOOLS/demo/cache.dir:myutil.demo.cache.longfunc(x,-delay)> MISS(18)


result: 36 (2.1sec)
Iteration 1 (42, 5)...

INFO:myutil.cache:Cache</home/andreoli/src/PYTOOLS/demo/cache.dir:myutil.demo.cache.longfunc(x,-delay)> MISS(19)


result: Exception[] (0.6sec)
Iteration 2 (36, 2)...

INFO:myutil.cache:Cache</home/andreoli/src/PYTOOLS/demo/cache.dir:myutil.demo.cache.longfunc(x,-delay)> HIT(18)


result: 36 (0.0sec)
Iteration 2 (42, 5)...

INFO:myutil.cache:Cache</home/andreoli/src/PYTOOLS/demo/cache.dir:myutil.demo.cache.longfunc(x,-delay)> MISS(20)


result: 42 (5.1sec)


In [13]:
longfunc.cache

## Processes

Restart the kernel before each run.

In [1]:
from myutil import loggingBasicConfig
loggingBasicConfig(level='INFO')
from myutil.demo.cache import proc, stepA, stepB
#[s.cache.clear() for s in stepA,stepB] # un/comment to clear/keep cache from previous run
e = proc()

In [2]:
e['abc']

INFO:myutil.cache:Cache</home/andreoli/src/PYTOOLS/demo/cache.dir:myutil.demo.cache.stepB(E,fr,to,r)> HIT(21)


14

In [3]:
stepB.cache

## Direct cache content display

In [4]:
from myutil.cache import CacheDB
db = CacheDB('cache.dir')
db